# Good hint Migration 
This notebook demonstrates how an LLM performs when presented with a Spring Security migration issue with a good example

Installing pre requisites and configuring Kai with `GPT-4o-mini`

In [ ]:
%pip uninstall kai -y
%pip install --no-cache-dir git+https://github.com/konveyor/kai.git@main
%pip install python-dotenv

In [26]:
from IPython.display import display, Markdown
from kai.llm_interfacing.model_provider import ModelProvider
from kai.kai_config import KaiConfigModels, SupportedModelProviders
from dotenv import load_dotenv
import os
load_dotenv(override=True) 

# Initialize the model provider
model = ModelProvider.from_config(KaiConfigModels(
    provider=SupportedModelProviders.CHAT_OPENAI,
    args={"model": "gpt-4o-mini"},
))

# Async rendering function for displaying the response
async def rendered_llm_call(prompt: str):
    response = await model.ainvoke_llm(prompt)
    display(Markdown(response.content))
    return response



Below is the snippet we are trying to migrate from springboot2 to springboot3. This code uses the discouraged `configure(AuthenticationManagerBuilder)` method inside a `WebSecurityConfigurerAdapter`, both of which are deprecated in Spring Security 5.7+ and removed in Spring Boot 3.

Our goal is to assess how well the LLM refactors this configuration without a hint, and whether it can replace deprecated patterns with the right approach.

In [27]:
before_code = """\
import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.authentication.builders.AuthenticationManagerBuilder;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.config.annotation.web.configuration.WebSecurityConfigurerAdapter;
import org.springframework.security.crypto.bcrypt.BCryptPasswordEncoder;
import org.springframework.security.crypto.password.PasswordEncoder;

@Configuration
@EnableWebSecurity
public class SecurityConfig extends WebSecurityConfigurerAdapter {
    @Override
    protected void configure(AuthenticationManagerBuilder auth) throws Exception {
        auth.inMemoryAuthentication()
            .withUser("user")
            .password(passwordEncoder().encode("password"))
            .roles("USER");
    }

    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }
}
"""

This rule flags the use of `configure(AuthenticationManagerBuilder)`. In Spring Boot 3, developers should instead use a `UserDetailsService` and `PasswordEncoder` bean for authentication configuration.

In [28]:
hint_to_migrate="""\
    
  description: configure(AuthenticationManagerBuilder) override is discouraged since Spring Security 5.7.
  message: |
    Overriding `configure(AuthenticationManagerBuilder)` is **discouraged** in Spring Security 5.7.
    Instead, define a `UserDetailsService` and `PasswordEncoder` bean.

    """

Similar example showing before and after migration

In [29]:
example_before_code="""\

@Configuration
public class SecurityConfiguration extends WebSecurityConfigurerAdapter {
    @Override
    protected void configure(AuthenticationManagerBuilder auth) throws Exception {
        UserDetails user = User.withDefaultPasswordEncoder()
            .username("user")
            .password("password")
            .roles("USER")
            .build();
        auth.inMemoryAuthentication()
            .withUser(user);
    }
}

"""

example_after_code="""\
@Configuration
public class SecurityConfiguration {
    @Bean
    public InMemoryUserDetailsManager userDetailsService() {
        UserDetails user = User.withDefaultPasswordEncoder()
            .username("user")
            .password("password")
            .roles("USER")
            .build();
        return new InMemoryUserDetailsManager(user);
    }
}
"""



example_before_code_2 = """\
@Configuration
@EnableWebSecurity
public class SecurityConfiguration extends WebSecurityConfigurerAdapter {

    @Override
    protected void configure(AuthenticationManagerBuilder auth) throws Exception {
        auth.inMemoryAuthentication()
            .withUser("testuser")
            .password(passwordEncoderBean().encode("testpass"))
            .roles("USER");
    }

    @Bean
    public PasswordEncoder passwordEncoderBean() {
        return new BCryptPasswordEncoder();
    }
}
"""

example_after_code_2 = """\
@Configuration
@EnableWebSecurity
public class SecurityConfiguration {

    @Bean
    public InMemoryUserDetailsManager inMemoryUserDetailsManager() {
        UserDetails testUser = User.builder()
            .username("testuser")
            .password(passwordEncoderBean().encode("testpass"))
            .roles("USER")
            .build();
        return new InMemoryUserDetailsManager(testUser);
    }

    @Bean
    public PasswordEncoder passwordEncoderBean() {
        return new BCryptPasswordEncoder();
    }
}
"""

example_before_code_3 = """\
@Configuration
@EnableWebSecurity
public class SecurityConfiguration extends WebSecurityConfigurerAdapter {

    @Override
    protected void configure(AuthenticationManagerBuilder auth) throws Exception {
        auth.inMemoryAuthentication()
            .withUser("admin")
            .password(passwordEncoder().encode("adminpass"))
            .roles("ADMIN");
    }

    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }
}
"""

example_after_code_3 = """\
@Configuration
@EnableWebSecurity
public class SecurityConfiguration {

    @Bean
    public InMemoryUserDetailsManager userDetailsService(PasswordEncoder passwordEncoder) {
        UserDetails adminUser = User.builder()
            .username("admin")
            .password(passwordEncoder.encode("adminpass"))
            .roles("ADMIN")
            .build();
        return new InMemoryUserDetailsManager(adminUser);
    }

    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }
}
"""




In [30]:

from kai.reactive_codeplanner.agent.reflection_agent import extract_ast_info, Language

original_summary = extract_ast_info(example_before_code, language=Language.Java)
original_summary_2 = extract_ast_info(example_before_code_2, language=Language.Java)
original_summary_3 = extract_ast_info(example_before_code_3, language=Language.Java)
updated_summary = extract_ast_info(example_after_code, language=Language.Java)
updated_summary_2 = extract_ast_info(example_after_code_2, language=Language.Java)
updated_summary_3 = extract_ast_info(example_after_code_3, language=Language.Java)

diff = original_summary.diff(updated_summary)
diff2 = original_summary_2.diff(updated_summary_2)
diff3 = original_summary_3.diff(updated_summary_3)

print(diff)
print(diff2)
print(diff3)

{'classes': {'modified': [{'name': 'SecurityConfiguration', 'super_class': {'old': 'extends WebSecurityConfigurerAdapter', 'new': ''}, 'fields': [], 'methods': {'added': [{'name': 'userDetailsService', 'parameters': '()', 'body': '{\n        UserDetails user = User.withDefaultPasswordEncoder()\n            .username("user")\n            .password("password")\n            .roles("USER")\n            .build();\n        return new InMemoryUserDetailsManager(user);\n    }', 'annotations': [{'name': 'Bean'}]}], 'removed': [{'name': 'configure', 'parameters': '(AuthenticationManagerBuilder auth)', 'body': '{\n        UserDetails user = User.withDefaultPasswordEncoder()\n            .username("user")\n            .password("password")\n            .roles("USER")\n            .build();\n        auth.inMemoryAuthentication()\n            .withUser(user);\n    }', 'annotations': [{'name': 'Override'}]}]}, 'annotations': [{'name': 'Configuration'}]}]}}
{'classes': {'modified': [{'name': 'Security

In [31]:
target_stack="springboot3"
prompt = f"""
You are assisting with migrating a Spring Boot 2 application to Spring Boot 3.

The code that needs migration is:

```java
{before_code}
```

You are given:

* A diff between a before/after successful **example** migrations.

* An hint that describes what should change.

Diff (use this as a guiding pattern for migration) - {diff} {diff2} {diff3}

Rule Hint - {hint_to_migrate}

Task
Refactor the given code {before_code} to make it compatible with Spring Boot 3 and Spring Security 6.

Ensure deprecated or removed APIs are replaced with valid alternatives.

1. Analyze:

Review the before-code, diff, summary, and hint carefully.

Identify and list any deprecated, removed, or outdated methods, APIs, libraries, or coding patterns present in the before-code.

Identify any missing constructs required for compatibility with {target_stack}.

2. Refactor:

Migrate the provided code fully to be compatible with {target_stack}.

Remove deprecated APIs and replace them with appropriate alternatives.

Follow {target_stack} best practices for dependency management, configuration, and structure.

Apply dependency injection principles if applicable (e.g., do not manually instantiate dependencies when framework-supported injection is possible).

Ensure the code is modular, clean, maintainable, and production-grade — not just minimal.

3. Validate:

Confirm that the refactored code is compilable and correct for {target_stack}.

Confirm that no deprecated methods, imports, or patterns remain.

Confirm that dependency injection, configuration, and new standards are applied correctly.

Confirm that imports or package references match the correct updated libraries.

4. Output:

Refactored Code in Java

Step-by-Step Migration Process

New Hint Summary: One-paragraph explanation of what changed and why (to guide future similar migrations).

"""

In [32]:

response = await rendered_llm_call(prompt)

### Step 1: Analyze

**Deprecated or Removed APIs:**
1. **WebSecurityConfigurerAdapter**: This class is deprecated in Spring Security 5.7 and removed in Spring Security 6. It is recommended to define a `UserDetailsService` and a `PasswordEncoder` bean instead.
2. **configure(AuthenticationManagerBuilder)**: Overriding this method is discouraged. Instead, we should define a `UserDetailsService` bean to manage user details.

**Missing Constructs for Compatibility:**
- The `SecurityFilterChain` bean should be defined to configure HTTP security.
- The `AuthenticationManager` bean should be defined to manage authentication.

### Step 2: Refactor

Here’s the refactored code:

```java
import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.web.builders.HttpSecurity;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.config.annotation.authentication.builders.AuthenticationManagerBuilder;
import org.springframework.security.crypto.bcrypt.BCryptPasswordEncoder;
import org.springframework.security.crypto.password.PasswordEncoder;
import org.springframework.security.core.userdetails.User;
import org.springframework.security.core.userdetails.UserDetails;
import org.springframework.security.core.userdetails.UserDetailsService;
import org.springframework.security.provisioning.InMemoryUserDetailsManager;
import org.springframework.security.web.SecurityFilterChain;

@Configuration
@EnableWebSecurity
public class SecurityConfig {

    @Bean
    public UserDetailsService userDetailsService(PasswordEncoder passwordEncoder) {
        UserDetails user = User.builder()
            .username("user")
            .password(passwordEncoder.encode("password"))
            .roles("USER")
            .build();
        return new InMemoryUserDetailsManager(user);
    }

    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }

    @Bean
    public SecurityFilterChain securityFilterChain(HttpSecurity http) throws Exception {
        http
            .authorizeRequests()
                .anyRequest().authenticated()
                .and()
            .formLogin()
                .and()
            .httpBasic();
        return http.build();
    }
}
```

### Step 3: Validate

- The refactored code is compilable and correct for Spring Boot 3 and Spring Security 6.
- No deprecated methods, imports, or patterns remain.
- Dependency injection is applied correctly, with `UserDetailsService` and `PasswordEncoder` defined as beans.
- The `SecurityFilterChain` is configured to manage HTTP security.

### Step 4: Output

**Refactored Code in Java:**
```java
import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.web.builders.HttpSecurity;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.crypto.bcrypt.BCryptPasswordEncoder;
import org.springframework.security.crypto.password.PasswordEncoder;
import org.springframework.security.core.userdetails.User;
import org.springframework.security.core.userdetails.UserDetails;
import org.springframework.security.core.userdetails.UserDetailsService;
import org.springframework.security.provisioning.InMemoryUserDetailsManager;
import org.springframework.security.web.SecurityFilterChain;

@Configuration
@EnableWebSecurity
public class SecurityConfig {

    @Bean
    public UserDetailsService userDetailsService(PasswordEncoder passwordEncoder) {
        UserDetails user = User.builder()
            .username("user")
            .password(passwordEncoder.encode("password"))
            .roles("USER")
            .build();
        return new InMemoryUserDetailsManager(user);
    }

    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }

    @Bean
    public SecurityFilterChain securityFilterChain(HttpSecurity http) throws Exception {
        http
            .authorizeRequests()
                .anyRequest().authenticated()
                .and()
            .formLogin()
                .and()
            .httpBasic();
        return http.build();
    }
}
```

**New Hint Summary:**
In migrating from Spring Boot 2 to Spring Boot 3, the `WebSecurityConfigurerAdapter` was removed, and the configuration now relies on defining a `UserDetailsService` and `PasswordEncoder` beans. Additionally, a `SecurityFilterChain` bean was introduced to manage HTTP security configurations. This approach aligns with the updated best practices in Spring Security, promoting a more modular and maintainable security configuration.

In [33]:
import re

# Extract the Java code block from the LLM response
def extract_java_code_from_response(markdown_text: str) -> str:
    code_blocks = re.findall(r"```java(.*?)```", markdown_text, re.DOTALL)
    return code_blocks[0].strip() if code_blocks else ""

# Example usage
refactored_java_code = extract_java_code_from_response(response.content)
print(refactored_java_code)

import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.web.builders.HttpSecurity;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.config.annotation.authentication.builders.AuthenticationManagerBuilder;
import org.springframework.security.crypto.bcrypt.BCryptPasswordEncoder;
import org.springframework.security.crypto.password.PasswordEncoder;
import org.springframework.security.core.userdetails.User;
import org.springframework.security.core.userdetails.UserDetails;
import org.springframework.security.core.userdetails.UserDetailsService;
import org.springframework.security.provisioning.InMemoryUserDetailsManager;
import org.springframework.security.web.SecurityFilterChain;

@Configuration
@EnableWebSecurity
public class SecurityConfig {

    @Bean
    public UserDetailsService userDetailsService(Pass

In [34]:
async def improve_hint(model, original_hint, before_code, diff):
    prompt = f"""
You are reviewing a migration rule.

Original Spring Boot 2 code:
```java
{before_code}
```

Migration Diff (JSON format):
```json
{diff}
{diff2}
{diff3}
```

Original Hint:
{original_hint}

Task:

- Analyze the before/after migration example carefully.

- Identify what was missing or vague in the original hint.

- Write an improved hint that:

    - Clearly states what elements should be changed.

    - Suggests correct alternatives.

    - If needed, includes important code patterns, best practices, or notes.


Format your output like:

- Improved Hint: (Print the updated hint with latest improvements)

- Migration Actions:

    - (Step 1)

    - (Step 2)

    - (Step 3)
"""
    result = await model.ainvoke_llm(prompt)
    return result.content.strip()



In [35]:
async def fix_code(model, before_code, refactored_code, improved_hint):
    prompt = f"""
You are assisting in migrating a Spring Boot 2 security config class.

Original code:
```java
{before_code}
```

Refactored attempt:
```java
{refactored_code}
```

Improved migration hint:
{improved_hint}

Task:
- Refactor the code to be valid for Spring Boot 3 and Spring Security 6.
- Remove deprecated APIs. 
- Ensure it uses correct bean definitions, imports, and compiles successfully.

Return only the updated code.
"""
    result = await model.ainvoke_llm(prompt)
    return result.content.strip()


In [36]:
import asyncio

async def get_improved_hint():
    improved = await improve_hint(model, hint_to_migrate, before_code, diff)
    print("Improved Hint:\n", improved)

    updated_code = await fix_code(model, before_code, refactored_java_code, improved)
    print("\n Updated Code:\n", updated_code)

await get_improved_hint()

Improved Hint:
 - Improved Hint: 
  description: Overriding `configure(AuthenticationManagerBuilder)` is discouraged since Spring Security 5.7. Instead, you should define a `UserDetailsService` bean and a `PasswordEncoder` bean to manage user authentication. 

  message: |
    To migrate from Spring Security 2 to 5.7, follow these steps:
    1. Remove the `configure(AuthenticationManagerBuilder)` method from your `SecurityConfiguration` class.
    2. Define a `UserDetailsService` bean that returns an `InMemoryUserDetailsManager` with your user details.
    3. Define a `PasswordEncoder` bean to handle password encoding.
    4. Use `User.withDefaultPasswordEncoder()` for quick setups or `User.builder()` with a `PasswordEncoder` for more secure configurations.

- Migration Actions:

    - Step 1: Remove the `configure(AuthenticationManagerBuilder auth)` method from the `SecurityConfiguration` class.

    - Step 2: Add a `UserDetailsService` bean method that creates and returns an `InMemor